In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import random
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing 
from xgboost import XGBRegressor
from datetime import datetime
from bayes_opt import BayesianOptimization
random.seed(1234)

# Define Functions

In [ ]:
# Performce cross validation using xgboost
def xgboostcv(X, y, fold, n_estimators, lr, depth, n_jobs, gamma, min_cw, subsample, colsample):
    uid = np.unique(fold)
    model_pred = np.zeros(X.shape[0])
    model_valid_loss = np.zeros(len(uid))
    model_train_loss = np.zeros(len(uid))
    for i in uid:
        x_valid = X[fold==i]
        x_train = X[fold!=i]
        y_valid = y[fold==i]
        y_train = y[fold!=i]
        model = XGBRegressor(n_estimators=n_estimators, learning_rate=lr, 
                           max_depth = depth, n_jobs = n_jobs, 
                           gamma = gamma, min_child_weight = min_cw,
                           subsample = subsample, colsample_bytree = colsample, random_state=1234)
        model.fit(x_train, y_train)

        pred = model.predict(x_valid)
        model_pred[fold==i] = pred
        model_valid_loss[uid==i] = mean_squared_error(y_valid, pred)
        model_train_loss[uid==i] = mean_squared_error(y_train, model.predict(x_train))
    return {'pred':model_pred, 'valid_loss':model_valid_loss, 'train_loss':model_train_loss}

# Compute MSE for xgboost cross validation
def xgboostcv_mse(n, p, depth, g, min_cw, subsample, colsample):
    model_cv = xgboostcv(X_train, y_train, fold_train, 
                         int(n)*100, 10**p, int(depth), n_nodes, 
                         10**g, min_cw, subsample, colsample)
    MSE = mean_squared_error(y_train, model_cv['pred'])
    return -MSE

# Display model performance metrics for each cv iteration
def cv_performance(model, y, fold):
    uid = np.unique(fold)
    pred = np.round(model['pred'])
    y = y.reshape(-1)
    model_valid_mse = np.zeros(len(uid))
    model_valid_mae = np.zeros(len(uid))
    model_valid_r2 = np.zeros(len(uid))
    for i in uid:
        pred_i = pred[fold==i]
        y_i = y[fold==i]
        model_valid_mse[uid==i] = mean_squared_error(y_i, pred_i)
        model_valid_mae[uid==i] = np.abs(pred_i-y_i).mean()
        model_valid_r2[uid==i] = r2_score(y_i, pred_i)
    
    results = pd.DataFrame(0, index=uid, 
                           columns=['valid_mse', 'valid_mae', 'valid_r2', 
                                    'valid_loss', 'train_loss'])
    results['valid_mse'] = model_valid_mse
    results['valid_mae'] = model_valid_mae
    results['valid_r2'] = model_valid_r2
    results['valid_loss'] = model['valid_loss']
    results['train_loss'] = model['train_loss']
    print(results)

# Display overall model performance metrics
def cv_overall_performance(y, y_pred):
    overall_MSE = mean_squared_error(y, y_pred)
    overall_MAE = (np.abs(y_pred-y)).mean()
    overall_RMSE = np.sqrt(np.square(y_pred-y).mean())
    overall_R2 = r2_score(y, y_pred)
    print("XGB overall MSE: %0.4f" %overall_MSE)
    print("XGB overall MAE: %0.4f" %overall_MAE)
    print("XGB overall RMSE: %0.4f" %overall_RMSE)
    print("XGB overall R^2: %0.4f" %overall_R2)    

# Plot variable importance
def plot_importance(model, columns):
    importances = pd.Series(model.feature_importances_, index = columns).sort_values(ascending=False)
    n = len(columns)
    plt.figure(figsize=(10,15))
    plt.barh(np.arange(n)+0.5, importances)
    plt.yticks(np.arange(0.5,n+0.5), importances.index)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.ylim([0,n])
    plt.gca().invert_yaxis()
    plt.savefig('variable_importance.png', dpi = 150)

# Save xgboost model
def save(obj, path):
    pkl_fl = open(path, 'wb')
    pickle.dump(obj, pkl_fl)
    pkl_fl.close()

# Load xgboost model
def load(path):
    f = open(path, 'rb')
    obj = pickle.load(f)
    f.close()
    return(obj)

# Parameter Values

In [ ]:
# Set a few values
validation_only = False # Whether test model with test data
n_nodes = 96 # Number of computing nodes used for hyperparamter tunning
trained = False # If a trained model exits
cols_drop = ['StationId', 'Date', 'PenRate', 'NumberOfLanes', 'Dir', 'FC', 'Month'] # Columns to be dropped
if trained:
    params = load('params.dat')
    xgb_cv = load('xgb_cv.dat')
    xgb = load('xgb.dat')

# Read Data

In [ ]:
if validation_only:
    raw_data_train = pd.read_csv("final_train_data.csv")
    data = raw_data_train.drop(cols_drop, axis=1)
    if 'Dir' in data.columns:
        data[['Dir']] = data[['Dir']].astype('category')
        one_hot = pd.get_dummies(data[['Dir']])
        data = data.drop(['Dir'], axis = 1)
        data = data.join(one_hot)
    if 'FC' in data.columns:
        data[['FC']] = data[['FC']].astype('category')
        one_hot = pd.get_dummies(data[['FC']])
        data = data.drop(['FC'], axis = 1)
        data = data.join(one_hot)
    week_dict = {"DayOfWeek": {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 
                               'Friday': 5, 'Saturday': 6, 'Sunday': 7}}
    data = data.replace(week_dict)

    X = data.drop(['Volume', 'fold'], axis=1)
    X_col = X.columns
    y = data[['Volume']]

    fold_train = data[['fold']].values.reshape(-1)
    X_train = X.values
    y_train = y.values
    
else:
    raw_data_train = pd.read_csv("final_train_data.csv")
    raw_data_test = pd.read_csv("final_test_data.csv")
    raw_data_test1 = pd.DataFrame(np.concatenate((raw_data_test.values, np.zeros(raw_data_test.shape[0]).reshape(-1, 1)), axis=1),
                                 columns = raw_data_test.columns.append(pd.Index(['fold'])))
    raw_data = pd.DataFrame(np.concatenate((raw_data_train.values, raw_data_test1.values), axis=0), 
                            columns = raw_data_train.columns)

    data = raw_data.drop(cols_drop, axis=1)
    if 'Dir' in data.columns:
        data[['Dir']] = data[['Dir']].astype('category')
        one_hot = pd.get_dummies(data[['Dir']])
        data = data.drop(['Dir'], axis = 1)
        data = data.join(one_hot)
    if 'FC' in data.columns:
        data[['FC']] = data[['FC']].astype('category')
        one_hot = pd.get_dummies(data[['FC']])
        data = data.drop(['FC'], axis = 1)
        data = data.join(one_hot)
    week_dict = {"DayOfWeek": {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 
                               'Friday': 5, 'Saturday': 6, 'Sunday': 7}}
    data = data.replace(week_dict)

    X = data.drop(['Volume'], axis=1)
    y = data[['Volume']]

    X_train = X.loc[X.fold!=0, :]
    fold_train = X_train[['fold']].values.reshape(-1)
    X_col = X_train.drop(['fold'], axis = 1).columns
    X_train = X_train.drop(['fold'], axis = 1).values
    y_train = y.loc[X.fold!=0, :].values

    X_test = X.loc[X.fold==0, :]
    X_test = X_test.drop(['fold'], axis = 1).values
    y_test = y.loc[X.fold==0, :].values

In [ ]:
# Explain variable names
X_name_dict = {'Temp': 'Temperature', 'WindSp': 'Wind Speed', 'Precip': 'Precipitation', 'Snow': 'Snow', 
               'Long': 'Longitude', 'Lat': 'Latitude', 'NumberOfLanes': 'Number of Lanes', 'SpeedLimit': 'Speed Limit', 
               'FRC': 'TomTom FRC', 'DayOfWeek': 'Day of Week', 'Month': 'Month', 'Hour': 'Hour', 
               'AvgSp': 'Average Speed', 'ProbeCount': 'Probe Count', 'Dir_E': 'Direction(East)', 
               'Dir_N': 'Direction(North)', 'Dir_S': 'Direction(South)', 'Dir_W': 'Direction(West)', 
               'FC_3R': 'FHWA FC(3R)', 'FC_3U': 'FHWA FC(3U)', 'FC_4R': 'FHWA FC(4R)', 'FC_4U': 'FHWA FC(4U)', 
               'FC_5R': 'FHWA FC(5R)', 'FC_5U': 'FHWA FC(5U)', 'FC_7R': 'FHWA FC(7R)', 'FC_7U': 'FHWA FC(7U)'}

In [ ]:
if validation_only == False:
    print(X_test.shape)

# Cross Validation & Hyperparameter Optimization

In [ ]:
# Set hyperparameter ranges for Bayesian optimization 
xgboostBO = BayesianOptimization(xgboostcv_mse,
                                 {'n': (1, 10),
                                  'p': (-4, 0),
                                  'depth': (2, 10),
                                  'g': (-3, 0),
                                  'min_cw': (1, 10), 
                                  'subsample': (0.5, 1), 
                                  'colsample': (0.5, 1)
                                 })

In [ ]:
# Use Bayesian optimization to tune hyperparameters
import time
start_time = time.time()
xgboostBO.maximize(init_points=10, n_iter = 50)
print('-'*53)
print('Final Results')
print('XGBOOST: %f' % xgboostBO.max['target'])
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Save the hyperparameters the yield the highest model performance
params = xgboostBO.max['params']
save(params, 'params.dat')
params

In [ ]:
# Perform cross validation using the optimal hyperparameters
xgb_cv = xgboostcv(X_train, y_train, fold_train, int(params['n'])*100, 
                   10**params['p'], int(params['depth']), n_nodes, 
                   10**params['g'], params['min_cw'], params['subsample'], params['colsample'])

In [ ]:
# Display cv results for each iteration
cv_performance(xgb_cv, y_train, fold_train)

In [ ]:
# Display overall cv results
cv_pred = xgb_cv['pred']
cv_pred[cv_pred<0] = 0
cv_overall_performance(y_train.reshape(-1), cv_pred)

In [ ]:
# Save the cv results
save(xgb_cv, 'xgb_cv.dat')

# Model Test

In [ ]:
# Build a xgboost using all the training data with the optimal hyperparameter
xgb = XGBRegressor(n_estimators=int(params['n'])*100, learning_rate=10**params['p'], max_depth = int(params['depth']), 
                   n_jobs = n_nodes, gamma = 10**params['g'], min_child_weight = params['min_cw'], 
                   subsample = params['subsample'], colsample_bytree = params['colsample'], random_state=1234)
xgb.fit(X_train, y_train)

In [ ]:
# Test the trained model with test data
if validation_only == False:
    y_pred = xgb.predict(X_test)
    y_pred[y_pred<0] = 0
    cv_overall_performance(y_test.reshape(-1), y_pred)

In [ ]:
# Plot variable importance
col_names = [X_name_dict[i] for i in X_col]
plot_importance(xgb, col_names)

In [ ]:
# Save the trained xgboost model
save(xgb, 'xgb.dat')

In [ ]:
# Produce cross validation estimates or estimates for test data
train_data_pred = pd.DataFrame(np.concatenate((raw_data_train.values, cv_pred.reshape(-1, 1)), axis=1),
                              columns = raw_data_train.columns.append(pd.Index(['PredVolume'])))
train_data_pred.to_csv('train_data_pred.csv', index = False)

if validation_only == False:
    test_data_pred = pd.DataFrame(np.concatenate((raw_data_test.values, y_pred.reshape(-1, 1)), axis=1),
                              columns = raw_data_test.columns.append(pd.Index(['PredVolume'])))
    test_data_pred.to_csv('test_data_pred.csv', index = False)


# Plot Estimations vs. Observations

In [ ]:
# Prepare data to plot estimated and observed values
if validation_only:
    if trained:
        plot_df = pd.read_csv("train_data_pred.csv")
    else:
        plot_df = train_data_pred
else:
    if trained:
        plot_df = pd.read_csv("test_data_pred.csv")
    else:
        plot_df = test_data_pred 
plot_df = plot_df.sort_values(by=['StationId', 'Date', 'Dir', 'Hour'])
plot_df = plot_df.set_index(pd.Index(range(plot_df.shape[0])))

In [ ]:
# Define a function to plot estimated and observed values for a day
def plot_daily_estimate(frc):
    indices = plot_df.index[(plot_df.FRC == frc) & (plot_df.Hour == 0)].tolist()
    from_index = np.random.choice(indices, 1)[0]
    to_index = from_index + 23
    plot_df_sub = plot_df.loc[from_index:to_index, :]
    time = pd.date_range(plot_df_sub.Date.iloc[0] + ' 00:00:00', periods=24, freq='H')
    plt.figure(figsize=(20,10))
    plt.plot(time, plot_df_sub.PredVolume, 'b-', label='XGBoost', lw=2)
    plt.plot(time, plot_df_sub.Volume, 'r--', label='Observed', lw=3)
    plt.tick_params(axis='both', which='major', labelsize=24)
    plt.ylabel('Volume (vehs/hr)', fontsize=24)
    plt.xlabel("Time", fontsize=24)
    plt.legend(loc='upper left', shadow=True, fontsize=24)
    plt.title('Station ID: {0}, MAE={1}, FRC = {2}'.format(
        plot_df_sub.StationId.iloc[0],
        round(np.abs(plot_df_sub.PredVolume-plot_df_sub.Volume).mean()),
        plot_df_sub.FRC.iloc[0]), fontsize=40)
    plt.savefig('frc_{0}.png'.format(frc), dpi = 150)
    return(plot_df_sub)

In [ ]:
# Define a function to plot estimated and observed values for a week
def plot_weekly_estimate(frc):
    indices = plot_df.index[(plot_df.FRC == frc) & (plot_df.Hour == 0) & (plot_df.DayOfWeek == 'Monday')].tolist()
    from_index = np.random.choice(indices, 1)[0]
    to_index = from_index + 24*7-1
    plot_df_sub = plot_df.loc[from_index:to_index, :]
    time = pd.date_range(plot_df_sub.Date.iloc[0] + ' 00:00:00', periods=24*7, freq='H')
    plt.figure(figsize=(20,10))
    plt.plot(time, plot_df_sub.PredVolume, 'b-', label='XGBoost', lw=2)
    plt.plot(time, plot_df_sub.Volume, 'r--', label='Observed', lw=3)
    plt.tick_params(axis='both', which='major', labelsize=24)
    plt.ylabel('Volume (vehs/hr)', fontsize=24)
    plt.xlabel("Time", fontsize=24)
    plt.legend(loc='upper left', shadow=True, fontsize=24)
    plt.title('Station ID: {0}, MAE={1}, FRC = {2}'.format(
        plot_df_sub.StationId.iloc[0],
        round(np.abs(plot_df_sub.PredVolume-plot_df_sub.Volume).mean()),
        plot_df_sub.FRC.iloc[0]), fontsize=40)
    plt.savefig('frc_{0}.png'.format(frc), dpi = 150)
    return(plot_df_sub)

In [ ]:
# Plot estimated and observed values for a day
frc2_daily_plot = plot_daily_estimate(2)
save(frc2_daily_plot, 'frc2_daily_plot.dat')

In [ ]:
# Plot estimated and observed values for a week
frc3_weekly_plot = plot_weekly_estimate(3)
save(frc3_weekly_plot, 'frc3_weekly_plot.dat')